In [1]:
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction
import cv2


In [2]:

# 1. Load your custom trained YOLO model
detection_model = AutoDetectionModel.from_pretrained(
    model_type='ultralytics',
    model_path='D:\Backup\WORK\MACH-3D\DefectClassification\models\yolov5m6u.pt', # Your trained model path
    confidence_threshold=0.3,
    device='cuda' # 'cpu' or 'cuda'
)

# 2. Run Sliced Inference
# This automatically chops the large image into 640x640 tiles, 
# detects defects, and merges the results.
results = get_sliced_prediction(
    r"D:\Backup\WORK\MACH-3D\Datasets\fdm-3d-printing-defect-Images\versions\1\FDM-3D-Printing-Defect-Dataset\data\Cracking\Image_20231128195537006_aug.jpg",
    detection_model,
    slice_height=500,
    slice_width=500,
    overlap_height_ratio=0.2, # 20% overlap ensures no defect is cut in half
    overlap_width_ratio=0.2
)

# 3. Visualize and Save
results.export_visuals(export_dir="demo_results/")

Performing prediction on 40 slices.


In [3]:
# 3. Extract Probabilities
for prediction in results.object_prediction_list:
    class_name = prediction.category.name
    score = prediction.score.value
    print(f" - {class_name}: {score:.2%}")

 - bird: 51.32%
 - tv: 39.58%
 - mouse: 41.43%
 - book: 52.69%
